In [1]:
from plot import *
from filter import *
from analysis import *
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# Files naming
graph_file = 'shinkansen_graph.pkl'
graph_file_weighted = 'shinkansen_graph_weighted.pkl'
graph_file_filtered = 'shinkansen_graph_filtered.pkl'
# Pre-processed data
shinkansen_join = 'data/shinkansen_join.csv'
# Output images
output_image = 'shinkansen_lines_weighted_visualization.png'
output_image_filtered = 'shinkansen_lines_visualization_filtered.png'
# Filter settings
min_degree = 2
top_n = 100

In [3]:
try:
    G = load_graph(graph_file_weighted)
    print('Graph loaded from file')
except:
    print('Graph file not found, creating graph')
    G = create_weighted_graph_form_csv(shinkansen_join)
    save_graph(G, graph_file)
    print('Graph saved to file')

Graph loaded from file


In [4]:
nodes_num = G.number_of_nodes()
edges_num = G.number_of_edges()
print(f'Number of nodes: {nodes_num}')
print(f'Number of edges: {edges_num}')

Number of nodes: 124
Number of edges: 122


In [6]:
# 计算平均度
degree = []
for node in G.nodes():
    degree.append(G.degree(node))
average_degree = sum(degree) / len(degree)
print(f'Average degree: {average_degree}')


Average degree: 1.967741935483871


In [10]:
print(len(find_shortest_path(G, '鹿児島中央', '新函館北斗')))

71


In [ ]:
# visualize_graph(G, output_image, size=15)
# print(f'Graph visualization saved to {output_image}')
visualize_weighted_graph(G, output_image, size=20, dpi=300, title='Shinkansen Lines', seed=21, layout='kamada')
print(f'Weighted Graph visualization saved to {output_image}')

In [ ]:
print(find_degree(G, '東京'))
print(find_degree(G, '新大阪'))

In [ ]:
print(find_all_path(G, '東京', '新大阪'))
print(find_shortest_path(G, '東京', '新大阪'))

In [ ]:
json_str = dict_to_json(find_all_communities(G))
with open('communities.json', 'w') as f:
    f.write(json_str)

In [ ]:
print(find_smallest_community(G))
print(find_largest_community(G))

In [18]:
node_to_id = {node: idx for idx, node in enumerate(G.nodes)}
id_to_node = {idx: node for node, idx in node_to_id.items()}
from infomap import Infomap
im = Infomap()
for u, v, data in G.edges(data=True):
    weight = data.get("weight", 1.0)  # 提取权重
    im.add_link(node_to_id[u], node_to_id[v], weight)\

im.run()
modules_dict = dict(im.modules)
community_result = {id_to_node[node_id]: module_id for node_id, module_id in modules_dict.items()}
# 打印社区划分结果
# print("社区划分结果:")
# for node, community in community_result.items():
#     print(f"节点 {node} 属于社区 {community}")

from collections import defaultdict
community_groups = defaultdict(list)
for node, community in community_result.items():
    community_groups[community].append(node)
output = {str(community): nodes for community, nodes in community_groups.items()}
output_json = json.dumps(output, ensure_ascii=False, indent=4)
with open('communities_infomap.json', 'w') as f:
    f.write(output_json)
